In [2]:
#import relevant libraries 
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import json
from pathlib import Path
import alpaca_trade_api as tradeapi
import os 
from dotenv import load_dotenv

%matplotlib inline

In [3]:
#engage API keys by activating .emv file
load_dotenv()

alpaca_api_key = os.getenv("Api_key")
alpaca_secret_key = os.getenv("Secret_key")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')


type(alpaca_api_key)

str

In [4]:
#load in historical data for S&P 500
sp500_df = api.alpha_vantage.historic_quotes('SPY', adjusted=True, output_format='pandas')

In [5]:
#DATA CLEANUP

#Select the column we need,  "adjusted close", and drop the others

sp500_df = sp500_df['5. adjusted close']
sp500_df.head()

date
2020-06-23    312.0500
2020-06-22    310.6200
2020-06-19    308.6400
2020-06-18    310.4060
2020-06-17    310.2865
Name: 5. adjusted close, dtype: float64

In [6]:
#Sort earliest to latest so that .pct_change() function works right.

sp500_df.sort_index(inplace=True, ascending=True)
sp500_df.head()

date
2000-06-22    99.5260
2000-06-23    98.6717
2000-06-26    99.9424
2000-06-27    99.2056
2000-06-28    99.4832
Name: 5. adjusted close, dtype: float64

In [15]:
#create a dataframe for the daily returns (pct_change) values and concat with SP500 close
returns_df = sp500_df.pct_change()
concat_returns = pd.concat([sp500_df, returns_df], axis="columns", join="inner")
concat_returns.head()

,5. adjusted close,5. adjusted close
date,,
2000-06-22,99.5260,NaN
2000-06-23,98.6717,-0.008584
2000-06-26,99.9424,0.012878
2000-06-27,99.2056,-0.007372
2000-06-28,99.4832,0.002798


In [16]:
#Change column names to avoid confusion
columns = ['S&P 500 close',
          'S&P 500 Daily Returns']
concat_returns.columns = columns
concat_returns.head()

,S&P 500 close,S&P 500 Daily Returns
date,,
2000-06-22,99.5260,NaN
2000-06-23,98.6717,-0.008584
2000-06-26,99.9424,0.012878
2000-06-27,99.2056,-0.007372
2000-06-28,99.4832,0.002798


In [17]:
# Drop nulls
concat_returns.dropna(inplace=True)

#count nulls again
concat_returns.isnull().sum()

S&P 500 close            0
S&P 500 Daily Returns    0
dtype: int64

In [18]:
# Check for duplicate rows
returns_df.duplicated()


date
2000-06-22    False
2000-06-23    False
2000-06-26    False
2000-06-27    False
2000-06-28    False
              ...  
2020-06-17    False
2020-06-18    False
2020-06-19    False
2020-06-22    False
2020-06-23    False
Name: 5. adjusted close, Length: 5032, dtype: bool

In [19]:
# drop duplicates
returns_df.drop_duplicates(inplace=True)
returns_df.head()

date
2000-06-22         NaN
2000-06-23   -0.008584
2000-06-26    0.012878
2000-06-27   -0.007372
2000-06-28    0.002798
Name: 5. adjusted close, dtype: float64